# Pointcloud rotor estimation

Consider the following challenge. We are presented with an input pointcloud $p_i$, and an output pointcloud $q_i = R[p_i] + \eta_i$, where $R$ is an unknown tranformation, and $\eta_i$ is Gaussian noise. The challenge is to reconstruct the transformation $R$.

In order to do this, we construct a symbolic tranformation $R$, whose entries are `symfit.Parameter` objects. We can then use `symfit` to find the rotor $R$.

In [1]:
%pip install -q kingdon symfit anywidget

Note: you may need to restart the kernel to use updated packages.


In [2]:
from kingdon import Algebra
from symfit import Fit, Model, CallableModel, Variable, Parameter, Eq, Mul
from symfit.core.minimizers import *
import numpy as np

We set up the number of points `n_points` in the pointcloud, the number of (Euclidean) dimensions of the modeling space `d`, and the standard deviation `sig` of the Gaussian distribution.

In [3]:
n_points = 10
d = 2
sig = 0.02

In [4]:
point_vals = np.zeros((d+1, n_points))
noise_vals = np.zeros((d+1, n_points))
point_vals[0, :] = np.ones(n_points)
point_vals[1:, :] = np.random.random((d, n_points))
noise_vals[1:, :] = np.random.normal(0.0, sig, (d, n_points))

In [5]:
alg = Algebra(d, 0, 1)
locals().update(alg.blades)

Create the points and noise as pseudovector of grade `d`.

In [6]:
noise = alg.vector(noise_vals).dual()
p = alg.vector(point_vals).dual()
p

[0.69331468 0.39322116 0.33057664 0.2689791  0.79561022 0.14113163
 0.07236445 0.94028488 0.5623913  0.20473193] 𝐞₀₁ + [-0.4461381  -0.6654981  -0.79724814 -0.83350406 -0.80097615 -0.70791822
 -0.82626575 -0.88341839 -0.81623028 -0.06027692] 𝐞₀₂ + [1. 1. 1. 1. 1. 1. 1. 1. 1. 1.] 𝐞₁₂

As the input rotor $R$, we use a translation by $0.5$ in the $\mathbb{e}_{20}$ direction, followed by a rotation around $\mathbb{e}_{12}$.

In [7]:
t = np.pi/3
T = alg.multivector(e=1, e02=-0.5)
S = alg.multivector(e=np.cos(t), e12=np.sin(t))
R = S*T
print(f'{T=!s}')
print(f'{S=!s}')
print(f'{R=!s}')

T=1 + -0.5 𝐞₀₂
S=0.5 + 0.866 𝐞₁₂
R=0.5 + -0.433 𝐞₀₁ + -0.25 𝐞₀₂ + 0.866 𝐞₁₂


We can now create the transformed pointcloud $q$, and visualize both both $p$ and $q$.

In [8]:
q = R.sw(p).grade(d) + noise

In [9]:
alg.graph(
    0xff0000, p, 'p', 
    0x0000ff, q, 'q', 
    0x000000, R.grade(2), 'axis',
    scale=1.0,
)

GraphWidget(cayley=[['1', 'e0', 'e1', 'e2', 'e01', 'e02', 'e12', 'e012'], ['e0', '0', 'e01', 'e02', '0', '0', …

We will now setup a symfit model to describe this transformation, where the rotor $R$ consists of `Parameter`'s, and the pointclouds $p$ and $q$ are symfit `Variable`'s.

In [10]:
R_par = alg.evenmv(name='R', symbolcls=Parameter)
p_var = alg.multivector(name='p', symbolcls=Variable, grades=(d,))
q_var = alg.multivector(name='q', symbolcls=Variable, grades=(d,))
print(R_par)
print(p_var)
print(q_var)

R + R01 𝐞₀₁ + R02 𝐞₀₂ + R12 𝐞₁₂
p01 𝐞₀₁ + p02 𝐞₀₂ + p12 𝐞₁₂
q01 𝐞₀₁ + q02 𝐞₀₂ + q12 𝐞₁₂


In [11]:
p_var_trans = (R_par >> p_var).filter()
# model = Model({q_var[k]: expr for k, expr in p_var_trans.items()})
model = Model({var: expr for var, expr in zip(q_var.values(), p_var_trans.values())})
model

Prepare the data for `symfit`.

In [12]:
datadict = {var.name: data for var, data in zip(p_var.values(), p.values())}
datadict.update({var.name: data for var, data in zip(q_var.values(), q.values())})

Initiate a `symfit.Fit` object with the model and data. We additionally supply the demand $R \widetilde{R} = 1$, since rotors should be normalized (i.e., othonormal transformations).

In [13]:
constraints = [
    Eq(R_par.normsq().e, 1)
]
fit = Fit(model, **datadict, constraints=constraints)

In [14]:
results = fit.execute()
print(results)


Parameter Value        Standard Deviation
R         5.034720e-01 1.691675e-02
R01       -4.331113e-01 1.072609e-02
R02       -2.500911e-01 2.333642e-02
R12       8.640116e-01 1.124685e-02
Status message         Optimization terminated successfully
Number of iterations   19
Objective              <symfit.core.objectives.LeastSquares object at 0x000001A07FEAF4F0>
Minimizer              <symfit.core.minimizers.SLSQP object at 0x000001A078B7D150>

Goodness of fit qualifiers:
chi_squared            0.012079077521410877
objective_value        0.006039538760705438
r_squared              0.9915089739480435

Constraints:
--------------------
Question: R**2 + R12**2 - 1 == 0?
Answer:   4.133138276074533e-12




`symfit` has used SLSQP because of the constraint, and we see that we have high accuracy on this constraint. Let us print the reconstructed rotor and it's norm. Furthermore, we can now apply $\widetilde{R}$ to $q$ to transform it back to the location of $p$ so we can visually inspect the quality of the reconstruction.

In [15]:
R_re = R_par(**results.params)
print(R_re)
print(R_re.normsq())

0.503 + -0.433 𝐞₀₁ + -0.25 𝐞₀₂ + 0.864 𝐞₁₂
1.0


In [16]:
p_reconstructed = (~R_re) >> q

In [17]:
from timeit import default_timer

def animate_q():
    """ Make cloud q rotate towards p. """
    s0 = R_re.grade(2).norm().e
    t0 = np.arctan2(s0, R_re.e)
    logR = R_re.grade(2) / s0
    t = t0 * (np.sin(default_timer() / 2) + 1 ) / 2 # [0, t0]
    
    R = np.cos(t) + logR * np.sin(t)
    return ~R >> q

alg.graph(
    0xff0000, p,
    0x0000ff, q, 'q', 
    0x880088, p_reconstructed, 'p reconstructed',
    0x000000, R_re.grade(2), 'reconst. axis',
    animate_q, 'q',
    animate=True,
    scale=1.0,
)

GraphWidget(cayley=[['1', 'e0', 'e1', 'e2', 'e01', 'e02', 'e12', 'e012'], ['e0', '0', 'e01', 'e02', '0', '0', …

We see that we have excellent agreement between the original and reconstructed pointclouds.